In [1]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt 
%matplotlib inline

import optuna
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import cross_val_score, cross_val_predict

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder,OrdinalEncoder

In [2]:
X = pd.read_csv('training_set_features.csv',index_col='respondent_id') 
y = pd.read_csv('training_set_labels.csv',index_col='respondent_id')
print(X.head(2))
print(y.head(2))

               h1n1_concern  h1n1_knowledge  behavioral_antiviral_meds  \
respondent_id                                                            
0                       1.0             0.0                        0.0   
1                       3.0             2.0                        0.0   

               behavioral_avoidance  behavioral_face_mask  \
respondent_id                                               
0                               0.0                   0.0   
1                               1.0                   0.0   

               behavioral_wash_hands  behavioral_large_gatherings  \
respondent_id                                                       
0                                0.0                          0.0   
1                                1.0                          0.0   

               behavioral_outside_home  behavioral_touch_face  \
respondent_id                                                   
0                                  1.0             

In [3]:
#indices are matching
np.testing.assert_array_equal(X.index.values,y.index.values)

In [4]:
print(X.shape)
print(y.shape)

(26707, 35)
(26707, 2)


In [5]:
for col in X.select_dtypes(include='object').columns.to_list():    
        print(col, X[col].unique())

age_group ['55 - 64 Years' '35 - 44 Years' '18 - 34 Years' '65+ Years'
 '45 - 54 Years']
education ['< 12 Years' '12 Years' 'College Graduate' 'Some College' nan]
race ['White' 'Black' 'Other or Multiple' 'Hispanic']
sex ['Female' 'Male']
income_poverty ['Below Poverty' '<= $75,000, Above Poverty' '> $75,000' nan]
marital_status ['Not Married' 'Married' nan]
rent_or_own ['Own' 'Rent' nan]
employment_status ['Not in Labor Force' 'Employed' 'Unemployed' nan]
hhs_geo_region ['oxchjgsf' 'bhuqouqj' 'qufhixun' 'lrircsnp' 'atmpeygn' 'lzgpxyit'
 'fpwskwrf' 'mlyzmhmf' 'dqpwygqj' 'kbazzjca']
census_msa ['Non-MSA' 'MSA, Not Principle  City' 'MSA, Principle City']
employment_industry [nan 'pxcmvdjn' 'rucpziij' 'wxleyezf' 'saaquncn' 'xicduogh' 'ldnlellj'
 'wlfvacwt' 'nduyfdeo' 'fcxhlnwr' 'vjjrobsf' 'arjwrbjb' 'atmlpfrs'
 'msuufmds' 'xqicxuve' 'phxvnwax' 'dotnnunm' 'mfikgejo' 'cfqqtusy'
 'mcubkhph' 'haxffmxo' 'qnlwzans']
employment_occupation [nan 'xgwztkwe' 'xtkaffoo' 'emcorrxb' 'vlluhbov' 'xqwwgdy

In [6]:
X.dtypes

h1n1_concern                   float64
h1n1_knowledge                 float64
behavioral_antiviral_meds      float64
behavioral_avoidance           float64
behavioral_face_mask           float64
behavioral_wash_hands          float64
behavioral_large_gatherings    float64
behavioral_outside_home        float64
behavioral_touch_face          float64
doctor_recc_h1n1               float64
doctor_recc_seasonal           float64
chronic_med_condition          float64
child_under_6_months           float64
health_worker                  float64
health_insurance               float64
opinion_h1n1_vacc_effective    float64
opinion_h1n1_risk              float64
opinion_h1n1_sick_from_vacc    float64
opinion_seas_vacc_effective    float64
opinion_seas_risk              float64
opinion_seas_sick_from_vacc    float64
age_group                       object
education                       object
race                            object
sex                             object
income_poverty           

In [7]:
numeric_features= X.select_dtypes(include='float64').columns.to_list()
numeric_features

['h1n1_concern',
 'h1n1_knowledge',
 'behavioral_antiviral_meds',
 'behavioral_avoidance',
 'behavioral_face_mask',
 'behavioral_wash_hands',
 'behavioral_large_gatherings',
 'behavioral_outside_home',
 'behavioral_touch_face',
 'doctor_recc_h1n1',
 'doctor_recc_seasonal',
 'chronic_med_condition',
 'child_under_6_months',
 'health_worker',
 'health_insurance',
 'opinion_h1n1_vacc_effective',
 'opinion_h1n1_risk',
 'opinion_h1n1_sick_from_vacc',
 'opinion_seas_vacc_effective',
 'opinion_seas_risk',
 'opinion_seas_sick_from_vacc',
 'household_adults',
 'household_children']

In [8]:
ordinal_features=['education','income_poverty']
categorical_features = [['< 12 Years','12 Years','Some College',
                         'College Graduate'],
                       ['Below Poverty','<= $75,000, Above Poverty',
                       '> $75,000']]

In [9]:
items = X.select_dtypes(include='object').columns.to_list()
nominal_features=[item for item in items if item not in ordinal_features]
nominal_features

['age_group',
 'race',
 'sex',
 'marital_status',
 'rent_or_own',
 'employment_status',
 'hhs_geo_region',
 'census_msa',
 'employment_industry',
 'employment_occupation']

In [10]:
#Preprocessing
numeric_pipeline = Pipeline([
    ('imputer',SimpleImputer(strategy='mean')),
    ('scaler',MinMaxScaler())
])

oridnal_pipeline = Pipeline([
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinal',OrdinalEncoder(categories=categorical_features))
    
])

nominal_pipeline = Pipeline([
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('onehot',OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num',numeric_pipeline,numeric_features),
    ('ord',oridnal_pipeline,ordinal_features),
    ('nom',nominal_pipeline,nominal_features)
])

In [11]:
X = X[numeric_features+ordinal_features+nominal_features]
X.columns 

Index(['h1n1_concern', 'h1n1_knowledge', 'behavioral_antiviral_meds',
       'behavioral_avoidance', 'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'health_insurance', 'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective',
       'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'household_adults',
       'household_children', 'education', 'income_poverty', 'age_group',
       'race', 'sex', 'marital_status', 'rent_or_own', 'employment_status',
       'hhs_geo_region', 'census_msa', 'employment_industry',
       'employment_occupation'],
      dtype='object')

In [12]:
print(X.index.name)

respondent_id


In [13]:
X_transformed = preprocessor.fit_transform(X)
X_transformed.shape

(26707, 100)

In [14]:
#train_test_split
y_combined = y['h1n1_vaccine'].astype(str)+'_'+y['seasonal_vaccine'].astype(str)
y_combined[:5]
# not using this

respondent_id
0    0_0
1    0_1
2    0_0
3    0_1
4    0_0
dtype: object

In [15]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,                                                 
                                                 stratify=y,
                                                random_state=42)

In [16]:
#Optuna
def objective(trial):
    params = {
        'learning_rate':trial.suggest_float('learning_rate',0.05,0.2),
        'n_estimators':trial.suggest_int('n_estimators',80,250),
        'subsample':trial.suggest_float('subsample',0.6,0.85),
        'min_samples_split':trial.suggest_int('min_samples_split',2,10),
        'max_depth':trial.suggest_int('max_depth',2,6),
        'max_features':trial.suggest_categorical('max_features',['sqrt','log2']),
        'random_state':42        
    }
    
    model_h1n1= Pipeline([
        ('preprocessing',preprocessor),
        ('classifier',GradientBoostingClassifier(**params,
        validation_fraction=0.1,
        n_iter_no_change=20))
    ])
    
    model_seasonal= Pipeline([
        ('preprocessing',preprocessor),
        ('classifier',GradientBoostingClassifier(**params,
        validation_fraction=0.1,
        n_iter_no_change=20))
    ])
    
    skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
    
    y_pred_h1n1 = cross_val_predict(model_h1n1, X_train,y_train['h1n1_vaccine'],
                                    cv=skf,method='predict_proba')
    y_pred_seasonal = cross_val_predict(model_seasonal, X_train,y_train['seasonal_vaccine'],
                                    cv=skf,method='predict_proba')
    
    roc_auc_h1n1 = roc_auc_score(y_train['h1n1_vaccine'],y_pred_h1n1[:,1])
    roc_auc_seasonal = roc_auc_score(y_train['seasonal_vaccine'],y_pred_seasonal[:,1])
    
    return (roc_auc_h1n1+roc_auc_seasonal)/2   

In [17]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

print('Best trial')
print('number: ',study.best_trial.number)
print('ROC_AUC: ',study.best_trial.value)
print('Best hyperparameters')
for key,value in study.best_params.items():
    print(key,value)    

[I 2025-06-24 19:27:25,673] A new study created in memory with name: no-name-20cdcdd8-6e98-4d1c-90ac-8478513023de
[I 2025-06-24 19:27:52,286] Trial 0 finished with value: 0.8557881185313685 and parameters: {'learning_rate': 0.07086263610246094, 'n_estimators': 95, 'subsample': 0.7469864454148583, 'min_samples_split': 2, 'max_depth': 3, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.8557881185313685.
[I 2025-06-24 19:28:18,766] Trial 1 finished with value: 0.8556159407560417 and parameters: {'learning_rate': 0.07950363614381538, 'n_estimators': 127, 'subsample': 0.8148443308690925, 'min_samples_split': 5, 'max_depth': 3, 'max_features': 'log2'}. Best is trial 0 with value: 0.8557881185313685.
[I 2025-06-24 19:28:49,293] Trial 2 finished with value: 0.8606646740562915 and parameters: {'learning_rate': 0.16525251632168797, 'n_estimators': 231, 'subsample': 0.8006059167179689, 'min_samples_split': 2, 'max_depth': 4, 'max_features': 'log2'}. Best is trial 2 with value: 0.86066467405

[I 2025-06-24 19:46:11,914] Trial 28 finished with value: 0.8643328148410816 and parameters: {'learning_rate': 0.05086599703791199, 'n_estimators': 204, 'subsample': 0.666483379217978, 'min_samples_split': 7, 'max_depth': 5, 'max_features': 'sqrt'}. Best is trial 28 with value: 0.8643328148410816.
[I 2025-06-24 19:47:00,316] Trial 29 finished with value: 0.8632876101949547 and parameters: {'learning_rate': 0.07625603953597554, 'n_estimators': 220, 'subsample': 0.6589363690368172, 'min_samples_split': 7, 'max_depth': 4, 'max_features': 'sqrt'}. Best is trial 28 with value: 0.8643328148410816.


Best trial
number:  28
ROC_AUC:  0.8643328148410816
Best hyperparameters
learning_rate 0.05086599703791199
n_estimators 204
subsample 0.666483379217978
min_samples_split 7
max_depth 5
max_features sqrt


In [18]:
best_params = study.best_params
print(study.best_params)

{'learning_rate': 0.05086599703791199, 'n_estimators': 204, 'subsample': 0.666483379217978, 'min_samples_split': 7, 'max_depth': 5, 'max_features': 'sqrt'}


In [19]:
# final model

final_model = Pipeline([
    ('preprocessing',preprocessor),
    ('classifier',MultiOutputClassifier(GradientBoostingClassifier(
    **best_params,
        validation_fraction=0.1,
        n_iter_no_change=20,
    random_state=42)))
])

In [20]:
final_model.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['h1n1_concern',
                                                   'h1n1_knowledge',
                                                   'behavioral_antiviral_meds',
                                                   'behavioral_avoidance',
                                                   'behavioral_face_mask',
                                                   'behavioral_wash_hands',
                                                   'behavioral_large_gatherings',
                                                   'behavio

In [21]:
test_proba = final_model.predict_proba(X_train)
h1n1_proba = test_proba[0][:,1]
seasonal_proba = test_proba[1][:,1]

h1n1_train_roc_auc = roc_auc_score(y_train['h1n1_vaccine'],h1n1_proba)
seasonal_train_roc_auc = roc_auc_score(y_train['seasonal_vaccine'],seasonal_proba)

print(f"h1n1_train_roc_auc: {h1n1_train_roc_auc:.4f}")
print(f"seasonal_train_roc_auc: {seasonal_train_roc_auc:.4f}")

h1n1_train_roc_auc: 0.8959
seasonal_train_roc_auc: 0.8845


In [22]:
test_proba = final_model.predict_proba(X_test)
h1n1_proba = test_proba[0][:,1]
seasonal_proba = test_proba[1][:,1]

h1n1_test_roc_auc = roc_auc_score(y_test['h1n1_vaccine'],h1n1_proba)
seasonal_test_roc_auc = roc_auc_score(y_test['seasonal_vaccine'],seasonal_proba)

print(f"h1n1_test_roc_auc: {h1n1_test_roc_auc:.4f}")
print(f"seasonal_test_roc_auc: {seasonal_test_roc_auc:.4f}")

h1n1_test_roc_auc: 0.8724
seasonal_test_roc_auc: 0.8625


In [ ]:
#Fit model with full
final_model.fit(X,y)

In [ ]:
test_proba = final_model.predict_proba(X)
h1n1_proba = test_proba[0][:,1]
seasonal_proba = test_proba[1][:,1]

h1n1_roc_auc = roc_auc_score(y['h1n1_vaccine'],h1n1_proba)
seasonal_roc_auc = roc_auc_score(y['seasonal_vaccine'],seasonal_proba)

print(f"h1n1_roc_auc: {h1n1_roc_auc:.4f}")
print(f"seasonal_roc_auc: {seasonal_roc_auc:.4f}")

In [ ]:
#Get test data from file
test_X = pd.read_csv('test_set_features.csv',index_col='respondent_id')
test_X = test_X[X.columns]

In [ ]:
#Predict for test data
test_preds = final_model.predict_proba(test_X)

h1n1_preds = test_preds[0][:,1]
seasonal_preds = test_preds[1][:,1]

In [ ]:
submission_df = pd.read_csv("./submission_format_gradientboost _240625.csv", 
                            index_col="respondent_id")

In [ ]:
# Make sure we have the rows in the same order
np.testing.assert_array_equal(test_X.index.values, 
                              submission_df.index.values)

In [ ]:
# Save predictions to submission data frame
submission_df["h1n1_vaccine"] = h1n1_preds
submission_df["seasonal_vaccine"] = seasonal_preds

submission_df.head()

In [ ]:
submission_df.to_csv('submission_format_gradientboost _240625.csv', index=True)

In [ ]:
df = pd.read_csv('submission_format_gradientboost _240625.csv')
print(df.head(5))
print(df.tail(5))